In [3]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

import os

import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pandas.io.json import json_normalize

import urllib.request
import json
import codecs
import facebook

InteractiveShell.ast_node_interactivity = "all"
matplotlib.rcParams['svg.fonttype'] = 'none'

In [78]:
from google2pandas import *

ImportError: cannot import name 'ServiceAccountCredentials'

In [28]:
token = 'EAACEdEose0cBAHccUgjgrDnLQ0pIZAlHU2Kvhy3CMOpwRvZAxYi1OXWZAjf672lHZAxridZCwyapaD7DpZC6lYTDZBh4t1UcVITEOEs7IvUuRBfwuP4TuDx9Eoc1BTU3Pe6kC3SCJjZCjEiYFZCwM6l0b5rr6Wiv9G0OH6z2pN9MsSE9DrIKJC0iHX7jQdYatvxoZD'
graph = facebook.GraphAPI(access_token=token, version='2.2')

In [63]:
## variables

kennzahl_shares = 66.8
kennzahl_pageviews = 2045.5
kennzahl_kommentare = 6.6
kennzahl_readtime = 186.6

## Setup metaInfo

In [64]:
week = 'KW 2'
id = '2'

In [66]:
metaInfo_current = pd.DataFrame(columns=['id', 'week', 'multisession', 'nutzer', 'published_articles', 
                                 'pageviews_news', 'pageviews_newswire', 'pageviews_total', 'facebook_interaktionen'])

metaInfo_new = ['', '', '', '', '', '', '', '', '']  
metaInfo_current.loc[len(metaInfo_current)] = metaInfo_new

In [67]:
metaInfo_current['week'] = week
metaInfo_current['id'] = id
metaInfo_current['multisession'] = 2726
metaInfo_current['nutzer'] = 78513

metaInfo_current

,id,week,multisession,nutzer,published_articles,pageviews_news,pageviews_newswire,pageviews_total,facebook_interaktionen
0,2,KW 2,2726,78513,,,,,


## Load and prepare google stats

In [68]:
## load google analyctics data

ga_stats = 'weekly_report/weekly_ga_stats_kw2.csv'

stats_weekly = pd.read_csv(ga_stats)

stats_weekly['url'] = stats_weekly['Seite'].str.split('/')

for i in range(0, len(stats_weekly['url'])):   
    try:
        stats_weekly.loc[i, 'article_id'] = stats_weekly['url'][i][4]
    except:
        stats_weekly.loc[i, 'article_id'] = ''
        
stats_weekly['article_id'] = pd.to_numeric(stats_weekly['article_id'], errors='coerce')
stats_weekly['Seitenaufrufe'] = pd.to_numeric(stats_weekly['Seitenaufrufe'], errors='coerce')

stats_weekly = stats_weekly.dropna()

stats_weekly.head()

,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,url,article_id
1,/de/2017_2/basel/739636,5214,4651,00:04:22,2074,"73,53 %","52,82 %","0,00 CHF","[, de, 2017_2, basel, 739636]",739636.0
2,/de/2017_2/leben/739595,4887,4538,00:04:34,3963,"91,42 %","83,14 %","0,00 CHF","[, de, 2017_2, leben, 739595]",739595.0
3,/de/2017_1/international/739238,4125,3651,00:05:25,3379,"86,36 %","81,82 %","0,00 CHF","[, de, 2017_1, international, 739238]",739238.0
4,/de/2017_2/basel/739842,4051,3778,00:03:09,2270,"87,80 %","66,90 %","0,00 CHF","[, de, 2017_2, basel, 739842]",739842.0
5,/de/2017_1/sport/739452,3945,3708,00:03:42,2224,"88,80 %","74,04 %","0,00 CHF","[, de, 2017_1, sport, 739452]",739452.0


In [70]:
stats_weekly['time'] = pd.DatetimeIndex(stats_weekly['Durchschn Besuchszeit auf Seite'])
stats_weekly['readtime'] = stats_weekly['time'].dt.minute * 60 + stats_weekly['time'].dt.second

In [71]:
stats_readtime = stats_weekly.groupby(by=['article_id']).readtime.mean()
stats_readtime = stats_readtime.to_frame(name = 'readtime')
stats_readtime = stats_readtime.reset_index()

stats_readtime.head()

,article_id,readtime
0,1.0,70.0
1,443.0,3.0
2,984.0,8.0
3,3915.0,41.0
4,3917.0,96.0


In [73]:
stats_pageviews = stats_weekly.groupby(by=['article_id']).Seitenaufrufe.sum()
stats_pageviews = stats_pageviews.to_frame(name = 'Seitenaufrufe')
stats_pageviews = stats_pageviews.reset_index()

stats_pageviews.head()

,article_id,Seitenaufrufe
0,1.0,24
1,443.0,2
2,984.0,3
3,3915.0,10
4,3917.0,96


In [74]:
stats = stats_pageviews.merge(stats_readtime, left_on='article_id', right_on='article_id')

## Get meta data of articles

In [75]:
def parse_articles(id):
    meta = pd.DataFrame(columns=['type', 'article_id'])
    url = 'http://www.tageswoche.ch/content-api/articles/'
    article_name = url + str(id)
    #print(id)
    try:
        df = pd.read_json(article_name, lines=True)
        meta['type'] = df['type']
        meta['published'] = df['published']
        meta['body'] = df['fields'][0]['body']
        meta['title'] = df['title']
        meta['url'] = df['url']
        meta['comments'] = df['comments_count']
    except:
        pass
    meta['article_id'] = id
    return meta

filepath = 'weekly_report/tawo_stats_meta_' + week + '.csv'

if os.path.exists(filepath):
    df = pd.read_csv(filepath, usecols=[1, 2, 3, 4, 5])
else:
    df = pd.concat([parse_articles(id) for id in stats['article_id']])

df.head()

,article_id,body,comments,published,title,type,url
0,443.0,"<p dir=""ltr"">Sie hetzen die Galerietreppe hoch...",0.0,2014-03-16T17:16:12+0100,Versalzener Zweitling,news,http://www.tageswoche.ch/de/2014_11/kultur/443...
0,3915.0,"<div>\r\n<h4 id=""internal-source-marker_0.8521...",0.0,2011-10-07T12:43:09+0200,AGB,static_page,http://www.tageswoche.ch/de/pages/about/3915/a...
0,3917.0,"<div style=""float: left; max-width: 430px; wid...",0.0,2011-10-07T12:43:59+0200,Impressum,static_page,http://www.tageswoche.ch/de/pages/about/3917/i...
0,3918.0,<p><strong><br /></strong></p>\r\n<p><strong>T...,0.0,2011-10-07T12:44:54+0200,Kontakt,static_page,http://www.tageswoche.ch/de/pages/about/3918/k...
0,3919.0,<p><strong>Das Wichtigste in aller Kürze:</str...,0.0,2011-10-07T12:44:43+0200,Community-Leitfaden,static_page,http://www.tageswoche.ch/de/pages/about/3919/c...


In [12]:
df['published'] = df['published'].str.extract('(\d\d\d\d-\d\d-\d\d)')

df = df[df.published != '0001-11-30']

df['published'] = pd.to_datetime(df['published'], format='%Y-%m-%d')

df.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


,article_id,published,title,type,url
0,443.0,2014-03-16,Versalzener Zweitling,news,http://www.tageswoche.ch/de/2014_11/kultur/443...
1,3915.0,2011-10-07,AGB,static_page,http://www.tageswoche.ch/de/pages/about/3915/a...
2,3917.0,2011-10-07,Impressum,static_page,http://www.tageswoche.ch/de/pages/about/3917/i...
3,3918.0,2011-10-07,Kontakt,static_page,http://www.tageswoche.ch/de/pages/about/3918/k...
4,3919.0,2011-10-07,Community-Leitfaden,static_page,http://www.tageswoche.ch/de/pages/about/3919/c...


In [9]:
df.to_csv(filepath)

In [13]:
# merge google stats and article meta data

df = df.merge(stats_weekly, left_on='article_id', right_on='article_id')

df.head()

,article_id,published,title,type,url,Seitenaufrufe
0,443.0,2014-03-16,Versalzener Zweitling,news,http://www.tageswoche.ch/de/2014_11/kultur/443...,2
1,3915.0,2011-10-07,AGB,static_page,http://www.tageswoche.ch/de/pages/about/3915/a...,10
2,3917.0,2011-10-07,Impressum,static_page,http://www.tageswoche.ch/de/pages/about/3917/i...,96
3,3918.0,2011-10-07,Kontakt,static_page,http://www.tageswoche.ch/de/pages/about/3918/k...,99
4,3919.0,2011-10-07,Community-Leitfaden,static_page,http://www.tageswoche.ch/de/pages/about/3919/c...,17


## Prepare subsets for news and newswire

In [14]:
df['type'] = df['type'].astype('category')

df_news = df[(df.type == 'news')]

df_news.head()

,article_id,published,title,type,url,Seitenaufrufe
0,443.0,2014-03-16,Versalzener Zweitling,news,http://www.tageswoche.ch/de/2014_11/kultur/443...,2
7,80382.0,2011-10-27,Die TagesWoche: Mehr als eine Zeitung,news,http://www.tageswoche.ch/de/2011_43/dialog/803...,3
8,80430.0,2011-10-28,Willkommen bei der TagesWoche!,news,http://www.tageswoche.ch/de/2011_43/basel/8043...,5
10,97670.0,2011-10-29,"Der Mann, der die Hölle von Schweizerhalle fot...",news,http://www.tageswoche.ch/de/2011_43/basel/9767...,5
11,104505.0,2011-11-04,"Adieu, Anarchie! Basels Feierszene wird flügge",news,http://www.tageswoche.ch/de/2011_44/kultur/104...,6


In [15]:
data_subset_news = df_news.set_index(['published'])

data_subset_news_week = data_subset_news.loc['2017-01-09':'2017-01-15']

In [16]:
# Save metainfos

metaInfo_current['pageviews_news'] = data_subset_news_week['Seitenaufrufe'].sum()

In [17]:
data_subset_news_week = data_subset_news_week.reset_index()

In [18]:
# Save metainfos

metaInfo_current['published_articles'] = data_subset_news_week['article_id'].nunique()

In [19]:
df_newswire = df[(df.type == 'newswire')]

data_subset_newswire = df_newswire.set_index(['published'])

data_subset_newswire_week = data_subset_newswire.loc['2017-01-09':'2017-01-15']

In [20]:
# Save metainfos

metaInfo_current['pageviews_newswire'] = data_subset_newswire_week['Seitenaufrufe'].sum()

In [21]:
# Save metainfos

metaInfo_current['pageviews_total'] = metaInfo_current['pageviews_news'] + metaInfo_current['pageviews_newswire']

## Get Facebook Stats

In [22]:
url = data_subset_news_week['url'][0]

In [23]:
post = graph.get_object(id=url)

post

{'id': 'http://www.tageswoche.ch/de/2017_1/schweiz/739248/warum-die-schweizer-steuerreform-entwicklungslaendern-schadet.htm',
 'og_object': {'description': 'Die Unternehmenssteurreform III wirkt sich nicht nur auf die Schweiz aus. Sie bietet neue Steuerschlupflöcher, die für Entwicklungsländer fatale Folgen haben.',
  'id': '1312789572128764',
  'title': 'Warum die Schweizer Steuerreform Entwicklungsländern schadet',
  'type': 'article',
  'updated_time': '2017-02-06T22:16:50+0000'},
 'share': {'comment_count': 0, 'share_count': 822}}

In [61]:
def get_facebook_shares(url):
    shares = pd.DataFrame(columns=['url', 'share_count'])
    shares_new = ['', '']  
    shares.loc[len(shares)] = shares_new
    try:
        post = graph.get_object(id=url)
        shares['share_count'] = post['share']['share_count']
        shares['url'] = url
    except:
        pass
        
    return shares

share_stats = pd.concat([get_facebook_shares(url) for url in data_subset_news_week['url']])

share_stats = share_stats.reset_index()
share_stats = share_stats[[1,2]]

share_stats.head()

,url,share_count
0,http://www.tageswoche.ch/de/2017_1/schweiz/739...,822
1,http://www.tageswoche.ch/de/2017_1/sport/73939...,2
2,http://www.tageswoche.ch/de/2017_1/basel/73939...,22
3,http://www.tageswoche.ch/de/2017_2/basel/73941...,0
4,http://www.tageswoche.ch/de/2017_2/kultur/7394...,0


In [62]:
share_stats

,url,share_count
0,http://www.tageswoche.ch/de/2017_1/schweiz/739...,822
1,http://www.tageswoche.ch/de/2017_1/sport/73939...,2
2,http://www.tageswoche.ch/de/2017_1/basel/73939...,22
3,http://www.tageswoche.ch/de/2017_2/basel/73941...,0
4,http://www.tageswoche.ch/de/2017_2/kultur/7394...,0
5,http://www.tageswoche.ch/de/2017_2/internation...,29
6,http://www.tageswoche.ch/de/2017_1/sport/73945...,8
7,http://www.tageswoche.ch/de/2017_2/basel/73945...,42
8,http://www.tageswoche.ch/de/2017_2/basel/73948...,0
9,http://www.tageswoche.ch/de/2017_2/kultur/7395...,311


In [19]:
# Show metainfos

metaInfo_old = pd.read_csv('weekly_report/metainfo.csv', usecols=[1,2,3,4,5,6,7])

metaInfo = metaInfo_old.append(metaInfo_current, ignore_index=True)

metaInfo_current

metaInfo.to_csv('weekly_report/metainfo.csv')

metaInfo

,id,week,multisession,nutzer,published_articles,pageviews_news,pageviews_newswire,pageviews_total
0,2,KW 2,2726,78513,53,73936,16377,90313


,id,multisession,nutzer,pageviews_news,pageviews_newswire,pageviews_total,published_articles,week
0,NaN,2640,90793,77117,25277,102394,57,KW 4
1,NaN,2581,93399,84454,16253,100707,53,KW 3
2,2,2726,78513,73936,16377,90313,53,KW 2


In [20]:
top5 = data_subset_news_week.sort_values('Seitenaufrufe', ascending=False).head(5)

In [21]:
top5['week'] = week

In [22]:
top5_series = (top5.groupby(['week'])
       .apply(lambda x: x[['Seitenaufrufe','article_id', 'title', 'url']].to_dict('r'))
       .rename(columns={0:'top'}))
top5_series

week
KW 2    [{'url': 'http://www.tageswoche.ch/de/2017_2/b...
dtype: object

In [23]:
flop5 = data_subset_news_week.sort_values('Seitenaufrufe', ascending=True).head(5)

In [24]:
flop5['week'] = week

In [25]:
flop5_series = (flop5.groupby(['week'])
       .apply(lambda x: x[['Seitenaufrufe','article_id', 'title', 'url']].to_dict('r'))
       .rename(columns={0:'flop'}))
flop5_series

week
KW 2    [{'url': 'http://www.tageswoche.ch/de/2017_2/s...
dtype: object

In [26]:
cols = "id week multisession	nutzer	published_articles	pageviews_news	pageviews_newswire	pageviews_total".split()
j = metaInfo_current[cols].groupby("week").first()
       #.to_json(orient='records', date_format='iso'))
j['top'] = top5_series
j['flop'] = flop5_series
j = j.reset_index()
j = j.to_json(orient='records')

In [27]:
with open('weekly_report/weekly_report.json', 'w') as fp:
    json.dump(json.loads(j), fp, indent=2, sort_keys=True)

print(json.dumps(json.loads(j), indent=2, sort_keys=True))

[
  {
    "flop": [
      {
        "Seitenaufrufe": 11,
        "article_id": 739739.0,
        "title": "Wie sehen Sie das, Herr K\u00fcnzli?",
        "url": "http://www.tageswoche.ch/de/2017_2/schweiz/739739/wie-sehen-sie-das-herr-kuenzli.htm"
      },
      {
        "Seitenaufrufe": 17,
        "article_id": 739526.0,
        "title": "Vom Natel zum Smartphone: Das GSM hat bald nichts mehr zu melden",
        "url": "http://www.tageswoche.ch/de/2017_2/leben/739526/vom-natel-zum-smartphone-das-gsm-hat-bald-nichts-mehr-zu-melden.htm"
      },
      {
        "Seitenaufrufe": 49,
        "article_id": 739568.0,
        "title": "Zehn Schritte zu weniger Abfall \u2013 und wo man sie (bald) anwenden kann",
        "url": "http://www.tageswoche.ch/de/2017_2/basel/739568/zehn-schritte-zu-weniger-abfall-und-wo-man-sie-bald-anwenden-kann.htm"
      },
      {
        "Seitenaufrufe": 59,
        "article_id": 739735.0,
        "title": "Wie wird das Tennisjahr 2017? Eine Auslegeordnung",


In [36]:
data_subset_topics = df.set_index(['published'])

data_subset_topics = data_subset_topics.loc['2017-01-23':'2017-01-29']

data_subset_topics = data_subset_topics.reset_index()

In [37]:
data_subset_topics

,published,article_id,title,type,url,Seitenaufrufe
0,2017-01-23,739808.0,Diese Schlafflüsterer helfen beim Eindösen,news,http://www.tageswoche.ch/de/2017_2/leben/73980...,290
1,2017-01-23,740339.0,Diese Firmen profitieren von der Steuerreform ...,news,http://www.tageswoche.ch/de/2017_3/basel/74033...,6949
2,2017-01-24,740358.0,Was tickt denn da?,news,http://www.tageswoche.ch/de/2017_3/kultur/7403...,210
3,2017-01-23,740404.0,Pino Bertelli: «Die Stadt erweitert sich – und...,news,http://www.tageswoche.ch/de/2017_3/kultur/7404...,2410
4,2017-01-23,740407.0,Nie mehr kalte Hände – dank der Schneeballzange,news,http://www.tageswoche.ch/de/2017_3/leben/74040...,3257
5,2017-01-27,740418.0,"Was zählt, ist was passiert – Lyrik im Netzwerk",news,http://www.tageswoche.ch/de/2017_3/kultur/7404...,1331
6,2017-01-24,740452.0,"Vier Gründe, warum die USR III vor allem den g...",news,http://www.tageswoche.ch/de/2017_3/basel/74045...,1913
7,2017-01-23,740465.0,Ja oder Nein? Antreten zum Erklärbär-Video – d...,news,http://www.tageswoche.ch/de/2017_3/basel/74046...,1406
8,2017-01-24,740481.0,So schlägt sich die obdachlose Lilian durch de...,news,http://www.tageswoche.ch/de/2017_3/basel/74048...,3647
9,2017-01-24,740483.0,Trump und wir: Wie wird das werden?,news,http://www.tageswoche.ch/de/2017_3/internation...,3079


In [38]:
def get_topics(id):
    result = pd.DataFrame(columns=['topic', 'article_id'])
    i = 0;
    url = 'http://www.tageswoche.ch/content-api/articles/' + str(id)
    try:
        response = urllib.request.urlopen(url)
        reader = codecs.getreader("utf-8")
        obj = json.load(reader(response))
        for i in range(0, len(obj['topics'])):
            result.loc[i] = obj['topics'][i]['title']
    except:
        result['topic'] = ''
        pass
    print(id)
    result['article_id'] = id
    return result #create a series

topics = pd.concat([get_topics(id) for id in data_subset_topics['article_id']])

topics

739808.0
740339.0
740358.0
740404.0
740407.0
740418.0
740452.0
740465.0
740481.0
740483.0
740631.0
740632.0
740633.0
740634.0
740635.0
740637.0
740638.0
740639.0
740640.0
740641.0
740642.0
740643.0
740644.0
740646.0
740647.0
740648.0
740649.0
740650.0
740651.0
740652.0
740654.0
740656.0
740657.0
740658.0
740659.0
740660.0
740662.0
740663.0
740664.0
740665.0
740666.0
740667.0
740669.0
740670.0
740671.0
740672.0
740674.0
740675.0
740676.0
740677.0
740678.0
740679.0
740681.0
740682.0
740683.0
740684.0
740685.0
740686.0
740687.0
740689.0
740690.0
740691.0
740692.0
740693.0
740694.0
740695.0
740696.0
740697.0
740698.0
740699.0
740700.0
740701.0
740702.0
740704.0
740705.0
740708.0
740709.0
740710.0
740711.0
740712.0
740713.0
740714.0
740716.0
740717.0
740718.0
740719.0
740720.0
740721.0
740722.0
740723.0
740725.0
740727.0
740728.0
740729.0
740730.0
740731.0
740732.0
740734.0
740735.0
740736.0
740737.0
740738.0
740739.0
740740.0
740741.0
740742.0
740743.0
740745.0
740746.0
740747.0
740748.0
7

,article_id,id,topic,topics
0,Digital,739808.0,Digital,NaN
1,Soziale Medien,739808.0,Soziale Medien,NaN
2,Internet,739808.0,Internet,NaN
0,Wirtschaft,740339.0,Wirtschaft,NaN
1,Pharmaindustrie,740339.0,Pharmaindustrie,NaN
2,Steuern,740339.0,Steuern,NaN
3,Unternehmenssteuer-Reform III,740339.0,Unternehmenssteuer-Reform III,NaN
0,Kultur,740358.0,Kultur,NaN
1,Musik,740358.0,Musik,NaN
2,Museen,740358.0,Museen,NaN


In [39]:
topics = topics[['topic', 'id']]

topics.head()

,topic,id
0,Digital,739808.0
1,Soziale Medien,739808.0
2,Internet,739808.0
0,Wirtschaft,740339.0
1,Pharmaindustrie,740339.0


In [40]:
topics = topics.reset_index()

topics

,index,topic,id
0,0,Digital,739808.0
1,1,Soziale Medien,739808.0
2,2,Internet,739808.0
3,0,Wirtschaft,740339.0
4,1,Pharmaindustrie,740339.0
5,2,Steuern,740339.0
6,3,Unternehmenssteuer-Reform III,740339.0
7,0,Kultur,740358.0
8,1,Musik,740358.0
9,2,Museen,740358.0


In [41]:
topics = topics[['topic', 'id']]

In [42]:
topics_stats = topics.merge(stats_weekly, left_on='id', right_on='article_id')

topics_stats.head()

,topic,id,article_id,Seitenaufrufe
0,Digital,739808.0,739808.0,290
1,Soziale Medien,739808.0,739808.0,290
2,Internet,739808.0,739808.0,290
3,Wirtschaft,740339.0,740339.0,6949
4,Pharmaindustrie,740339.0,740339.0,6949


In [43]:
pageviews_topics = topics_stats.groupby(by=['topic'])['Seitenaufrufe'].sum().sort_values(ascending=False)

In [44]:
pageviews_topics

topic
Kanton Basel-Stadt               16478
Sport                            15107
Region Basel                     14936
Wirtschaft                       12015
Wetter                            9829
Fussball                          9454
FC Basel                          9454
Unternehmenssteuer-Reform III     9185
Steuern                           8862
Klimawandel                       8037
Verkehr                           7996
Pharmaindustrie                   7848
Baselland                         7232
Auch das noch                     7232
Tennis                            7179
Basel                             6759
Basler Zeitung                    6072
Roger Federer                     5523
Hans-Peter Wessels                5461
Wohnen                            4948
Sozialwesen                       4400
Sozialwerke                       4400
Fotografie                        4202
Gesellschaft                      4115
Stadtentwicklung                  3711
Arbeitslosigkeit   